In [1]:
from src_clean.dataloader.dataloader import DataLoader
from src_clean.preprocessor.prepro_new import PreprocessorTwo

In [2]:
train_df, test_df = DataLoader.load_train_test_dfs()
test_df = PreprocessorTwo.prepro_comp_columns(test_df)
train_df, test_df = PreprocessorTwo.calculate_statistics(train_df, test_df)
train_df, test_df = PreprocessorTwo.mean_pos_per_prop_id(train_df, test_df)
test_df = PreprocessorTwo.drop_cols(test_df)
train_df, test_df = PreprocessorTwo.impute(train_df, test_df)

In [5]:
from src_clean.enselmble.ensemble import Ensemble
from joblib import load
# Load pretrained models
lambda_mart_model = load('../lambda_mart.joblib')
catboost_model = load('../catboost/catboost.joblib')
#bayesian_ridge_model = load('../bayesian_ridge/bayesian_ridge.joblib')

model_data_list = [(lambda_mart_model, test_df.to_numpy()), (catboost_model, test_df.to_numpy())]

ensemble_preds = Ensemble.ensemble_predictions(model_data_list)

([(LGBMRanker(learning_rate=0.12, metric='ndcg', num_iterations=400,
           objective='lambdarank'), array([[-1.        , -1.        ,  3.        , ...,  2.        ,
         0.        , 17.59677419],
       [-1.        , -1.        ,  3.        , ...,  2.        ,
         0.        , 17.16393443],
       [-1.        , -1.        ,  2.        , ...,  2.        ,
         0.        , 17.30201342],
       ...,
       [-1.        , -1.        ,  4.        , ...,  2.        ,
         0.        , 20.33333333],
       [-1.        , -1.        ,  4.        , ...,  2.        ,
         1.        , 13.66666667],
       [-1.        , -1.        ,  4.        , ...,  1.        ,
         0.        , 11.38461538]])), (<catboost.core.CatBoostRegressor object at 0x2b7a5baf0>, array([[-1.        , -1.        ,  3.        , ...,  2.        ,
         0.        , 17.59677419],
       [-1.        , -1.        ,  3.        , ...,  2.        ,
         0.        , 17.16393443],
       [-1.        , -

In [6]:
from src_clean.score.ndcg import calculate_ndcg
from src_clean.ranker.ranker import Ranker

ranker = Ranker()
ranking_file_path = ranker.make_ranking_from_prediction(
    ensemble_preds, model_name="Ensemble"
)
calculate_ndcg(ranking_file_path)

0.40833511920137955